<a href="https://colab.research.google.com/github/amandatz/computational-linear-algebra/blob/main/Atividade6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 6

Amanda Topanotti Zanette (22100776)

**Importações e funções auxiliares**

In [1]:
using LinearAlgebra
using Plots

Esse trecho de código é usado apenas para garantir a repetibilidade dos números aleatórios gerados.

In [2]:
using Random
Random.seed!(123)

TaskLocalRNG()

### Questão 1

In [3]:
function gauss_seidel(A, b; max_iter=1000, tol=1e-6)
    n = size(A, 1)
    x = zeros(n)
    x_new = copy(x)

    for k in 1:max_iter
        for i in 1:n
            sum1 = sum(A[i,j] * x_new[j] for j in 1:i-1)
            sum2 = sum(A[i,j] * x[j] for j in i+1:n)
            x_new[i] = (b[i] - sum1 - sum2) / A[i,i]
        end

        residual = norm(A * x_new - b)
        if residual < tol
            println("Convergência alcançada após $k iterações")
            return x_new
        end

        x = copy(x_new)
    end

    println("Máximo de iterações alcançado sem convergência")
    return x
end

gauss_seidel (generic function with 1 method)

In [4]:
function gradientes_conjugados(A, b; max_iter=1000, tol=1e-6)
    n = size(A, 1)
    x = zeros(n)
    r = b - A * x
    p = copy(r)

    for k in 1:max_iter
        Ap = A * p
        alpha = dot(r, r) / dot(p, Ap)
        x = x + alpha * p
        r_new = r - alpha * Ap

        if norm(r_new) < tol
            println("Convergência alcançada após $k iterações")
            return x
        end

        beta = dot(r_new, r_new) / dot(r, r)
        p = r_new + beta * p
        r = r_new
    end

    println("Máximo de iterações alcançado sem convergência")
    return x
end

gradientes_conjugados (generic function with 1 method)

### Questão 2

### Questão 3

### Questão 4